In [2]:
import numpy as np 
import time 
import torch 
import torch.nn as nn
import torch.nn.functional as F 
import dgl 
from dgl.data import CoraGraphDataset
from sklearn.metrics import f1_score 

In [4]:
dropout=0.5 
Lr=1e-2
epoch=50
hd=128
nlayers=2 
wd=5e-4
aggre='gcn'
# 하이퍼 파라미터 값 설정 


In [5]:
G=CoraGraphDataset() 
numClasses=G.num_classes
G=G[0]
#노드들의 feauture와 dimention 
features=G.ndata['feat']
inFD=features.shape[1]
#각 노드들의 실제 라벨 
labels=G.ndata['label']
#학습/테스트 사용할 때 노드들에 대한 표시 
trainmask=G.ndata['train_mask']
testmask=G.ndata['test_mask']

print(G)

print(features)
print(inFD)
print(labels)


  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'train_mask': Scheme(shape=(), dtype=torch.bool), 'label': Scheme(shape=(), dtype=torch.int64), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'feat': Scheme(shape=(1433,), dtype=torch.float32)}
      edata_schemes={})
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
1433
tensor([3, 4, 4,  ..., 3, 3, 3])


In [7]:
def evluateTrain(model, features, lables, mask):
    model.eval()
    with torch.no_grad():
        logits=model(features)
        logits=logits[mask]
        labels=labels[mask]
        _,indices =torch.max(logits,dim=1)
        correct= torch.sum(indices==labels)
        return correct.item()*1.0/len(labels)

def evaluateTest(mode, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits=model(features)
        logits=logits[mask]
        labels=labels[mask]
        _,indices =torch.max(logits, dim=1)

        macro_f1=f1_score(labels,indices,average='macro')
        correct= torch.sum(indices==labels)
        return correct.item()*1.0/len(labels),macro_f1


In [8]:
def train(model, lossfunction,features,labels,trainmask,optimizer,epoches):
    executionTime=list()

    for epochs in range(epoches):
        model.train()

        starttime=time.time()
        logits=model(features)
        loss=lossfunction(logits[trainmask],labels[trainmask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        executionTime.append(time.time()-starttime)

        acc=evluateTrain(model,features,labels,trainmask)

        print(f'epoch:{epoches} Time :{np.mean(executionTime)} Loss:{loss.item()} ACC:{acc}')

def test(model,features,labels,testmask):
    acc,macro_f1=evaluateTest(model,features,labels,testmask)
    print('Test ACC{:.4f}'.format(acc))
    print('Test macro_f1 {:.4f}'.format(macro_f1)
    )




In [16]:
from dgl.nn.pytorch.conv import SAGEConv
'''
dropout=0.5 
Lr=1e-2
epoch=50
hd=128
nlayers=2 
wd=5e-4
aggre='gcn'
'''
class GraphSAGE(nn.Module):
    def __init__(self,graph, infeatDIM,numHiddenDim,numClasses,numLayers,activationFuntion,dropoutProb,aggregatorType):
        super(GraphSAGE,self).__init__()
        self.layers=nn.ModuleList()
        self.graph=graph

        #인풋 레이어 
        self.layers.append(SAGEConv(infeatDIM,numHiddenDim,aggregatorType,dropoutProb,activationFuntion))
        #히든레이어 
        for i in range(numLayers):
            self.layers.append(SAGEConv(numHiddenDim,numHiddenDim,aggregatorType,dropoutProb,activationFuntion))
        self.layers.append(SAGEConv(numHiddenDim,numClasses,aggregatorType,dropoutProb,activation=None))
    

    def forward(self, features):
        x=features
        for layer in self.layers:
            x=layer(self.graph,x)
        return x

In [17]:
'''
dropout=0.5 
Lr=1e-2
epoch=50
hd=128
nlayers=2 
wd=5e-4
aggre='gcn'
'''
model= GraphSAGE(G,inFD,hd,numClasses,nlayers,F.relu,dropout,aggre)
print(model)

loss=torch.nn.CrossEntropyLoss()

# 옵티마이저 초기화 
optimizer=torch.optim.Adam(model.parameters(),lr=Lr,weight_decay=wd)


GraphSAGE(
  (layers): ModuleList(
    (0): SAGEConv(
      (feat_drop): Dropout(p=0.5, inplace=False)
      (fc_neigh): Linear(in_features=1433, out_features=128, bias=True)
    )
    (1): SAGEConv(
      (feat_drop): Dropout(p=0.5, inplace=False)
      (fc_neigh): Linear(in_features=128, out_features=128, bias=True)
    )
    (2): SAGEConv(
      (feat_drop): Dropout(p=0.5, inplace=False)
      (fc_neigh): Linear(in_features=128, out_features=128, bias=True)
    )
    (3): SAGEConv(
      (feat_drop): Dropout(p=0.5, inplace=False)
      (fc_neigh): Linear(in_features=128, out_features=7, bias=True)
    )
  )
)
